In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import holidays as pyholidays

In [15]:
shift_type={"morning":["M", "COM", "DISP"],"afternoon":["T1", "T2", "CHX"]}

# demands dictionary has a first key representing the shift type, a second key representing the day of the week, and a third  separating
# holidays and not holidays using 1 to indicate holiday and 0 to indicate not holiday

def random_demands_generator(shift_type,year:int=2022,month:int=5):

    demands={}
    for shift in shift_type.keys():
        morning_max_demand=np.random.randint(2,5)
        afternoon_max_demand=np.random.randint(1,morning_max_demand)
        for st in shift_type[shift]:
            for day in range(7):
                for holiday in [0,1]:
                    if shift=="morning":
                        if st=="M":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-1
                        elif st=="COM":
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=morning_max_demand-1
                            else:
                                demands[(st,day,holiday)]=morning_max_demand-2
                        else:
                            if day <5 and holiday==0:
                                demands[(st,day,holiday)]=0
                            else:
                                demands[(st,day,holiday)]=max(morning_max_demand-2,1)
                    if shift=="afternoon":
                        if st=="T1":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=afternoon_max_demand
                            else:
                                demands[(st,day,holiday)]=afternoon_max_demand
                        elif st=="T2":
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,1)
                            else:
                                demands[(st,day,holiday)]=max(afternoon_max_demand-1,0)
                        else:
                            if day<5 and holiday==0:
                                demands[(st,day,holiday)]=max(morning_max_demand-1,1)  
                            else:
                                demands[(st,day,holiday)]=0                  
    list_shifts=[]
    # list of dates of january 2023
    
    days_in_month = None
    if month == 2:
        days_in_month = 28
    elif month in [4, 6, 9, 11]:
        days_in_month = 30
    else:
        days_in_month = 31
        

    dates = pd.date_range(start=f'{month}/1/{year}', end=f'{month}/{days_in_month}/{year}')
    shift_id=0
    for fecha in dates:
        for shift in shift_type:
            for jornada in shift_type[shift]:
                list_shifts.append([shift_id, 
                                    fecha.strftime("%Y-%m-%d"), 
                                    fecha.week,
                                    jornada, shift,fecha.weekday(),
                                    fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023),
                                    demands[(jornada,fecha.weekday(),int( fecha.strftime("%Y-%m-%d") in pyholidays.CO(years=2023)))]
                                    ])
                shift_id+=1
    df_shifts=pd.DataFrame(list_shifts,columns=["shift_id","shift_date","week_of_year","shift","shift_type","weekday","holiday","demand"])
    df_shifts.index=df_shifts["shift_id"]
    
    return df_shifts



In [12]:
import numpy as np
import calendar
import datetime
def create_random_instance(n_nurses:int,year:int=2022,month:int=5,seed:int=None):
    """
    Create a random instance of the nurse scheduling problem.

    return: pandas dataframe with columns 'nurse_id', 'nurse_name',  'shift_preference',
       'morning_availability_labor_day', 'morning_availability_weekend',
       'afternoon_availability_labor_day', 'afternoon_availability_weekend',
       'dates_off', 'accumulated_hours','vacations'
    """
    if seed != None:
        np.random.seed(seed)
    
    nurse_id = np.arange(n_nurses)
    nurse_name = np.array([f'Nurse {i}' for i in nurse_id])
    shift_preference = ['afternoon' if np.random.rand() < 0.5 else 'morning' for i in nurse_id]
    morning_availability_labor_day = np.random.randint(0,2,n_nurses)
    morning_availability_weekend = np.random.randint(0,2,n_nurses)
    afternoon_availability_labor_day = np.random.randint(0,2,n_nurses)
    afternoon_availability_weekend = np.random.randint(0,2,n_nurses)


    #enumerate weekend days of the month
    weekend_days = []
    for i in range(1,calendar.monthrange(year,month)[1]+1):
        if datetime.date(year,month,i).weekday() > 4:
            weekend_days.append(i)
    
    # select random weekend days
    dates_off = []
    for i in range(n_nurses):
        dates_off.append(np.random.choice(weekend_days, size=np.random.randint(0,len(weekend_days)), replace=False))
    accumulated_hours = list(np.random.randint(-30,30,n_nurses))
    vacations = []
    for i in range(n_nurses):
        if np.random.rand() < 0.2:
            vacations_duration = np.random.randint(7,15)
            start_date = np.random.randint(1,calendar.monthrange(year,month)[1]+1)
            vacations.append([start_date+j for j in range(vacations_duration) if start_date+j<32])
        else:
            vacations.append([])
    return pd.DataFrame({'nurse_id':nurse_id,
                            'nurse_name':nurse_name,
                            'shift_preference':shift_preference,
                            'morning_availability_labor_day':morning_availability_labor_day,
                            'morning_availability_weekend':morning_availability_weekend,
                            'afternoon_availability_labor_day':afternoon_availability_labor_day,
                            'afternoon_availability_weekend':afternoon_availability_weekend,
                            'dates_off':dates_off,
                            'accumulated_hours':accumulated_hours,
                            'vacations':vacations})



In [16]:
import os
from os.path import exists
from app.libs.HHCRS_multiobjective_model import check_feasibility
from itertools import product

for iter in range(10):
    for tot_nurses,month,year in product(range(10,21),range(1,13),[2022,2023]):

        df_shifts=random_demands_generator(shift_type,year,month)
        df_random=create_random_instance(tot_nurses,year,month)
        df_random.to_json('tmp/nurses.json',orient='records')
        df_shifts.to_json('tmp/shifts.json',orient='records')

        if check_feasibility('tmp/') == "Optimal":

            # checks whether the folder name already exists, otherwise creates it
            if not exists(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}'):
                os.mkdir(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}')

            # checks the number of files in the folder and creates a new one with the next number
            files = os.listdir(f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}')
            if len(files) == 0:
                nurse_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/n_nurses_{tot_nurses}_{month}-{year}_1.json'
                shifts_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/shifts_{month}-{year}_1.json'
            else:
                nurse_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/n_nurses_{tot_nurses}_{month}-{year}_{len(files)+1}.json'
                shifts_file = f'instances/n_nurses_{tot_nurses}_m_{month}_y_{year}/shifts_{month}-{year}_{len(files)+1}.json'

            df_random.to_json(nurse_file,orient='records')
            df_shifts.to_json(shifts_file,orient='records')
            #  move to a new folder named as the details of the instance
    
    
print()

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/0f505ee8849a4d179a5d237043ad7c34-pulp.lp
Reading time = 0.01 seconds
OBJ: 3810 rows, 2196 columns, 16389 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3810 rows, 2196 columns and 16389 nonzeros
Model fingerprint: 0x9a1de522
Variable types: 32 continuous, 2164 integer (2164 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 1987 rows and 711 columns
Presolve time: 0.03s
Presolved: 1

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/04b00640c7984827ada614e0dae603ab-pulp.lp
Reading time = 0.01 seconds
OBJ: 3870 rows, 2268 columns, 16607 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3870 rows, 2268 columns and 16607 nonzeros
Model fingerprint: 0x17860669
Variable types: 32 continuous, 2236 integer (2236 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 2061 rows and 626 columns
Presolve time: 0.02s
Presolved: 1

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


...

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/5545c5d0e38d468eb5df291dd07ec66e-pulp.lp
Reading time = 0.02 seconds
OBJ: 7029 rows, 4185 columns, 30991 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7029 rows, 4185 columns and 30991 nonzeros
Model fingerprint: 0x7a787127
Variable types: 59 continuous, 4126 integer (4126 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 3746 rows and 1322 columns
Presolve time: 0.04s
Presolved: 

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Set parameter Username
Set parameter LogFile to value "gurobi.log"
Academic license - for non-commercial use only - expires 2024-04-04
Using license file /Users/user/gurobi.lic

Gurobi Optimizer version 10.0.1 build v10.0.1rc0 (mac64[x86])
Copyright (c) 2023, Gurobi Optimization, LLC

Read LP format model from file /var/folders/xc/t5gbnt3n5sq5bkg65xw9f1g00000gn/T/b2417e03c5784d319aebdcdd0a7f052d-pulp.lp
Reading time = 0.02 seconds
OBJ: 7038 rows, 4191 columns, 31002 nonzeros

CPU model: Intel(R) Core(TM) i7-6820HQ CPU @ 2.70GHz
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 7038 rows, 4191 columns and 31002 nonzeros
Model fingerprint: 0xad16afdc
Variable types: 59 continuous, 4132 integer (4132 binary)
Coefficient statistics:
  Matrix range     [1e+00, 8e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+02]
Presolve removed 3759 rows and 1324 columns
Presolve time: 0.04s
Presolved: 

/Users/user/.pyenv/versions/3.9.16/lib/python3.9/site-packages/pulp/pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
